<a href="https://colab.research.google.com/github/TokyoYoshida/GoogleColab/blob/master/playground_deep_learning_keiba2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# 必要ならば以下のようにディレクトリ移動する
 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/'My Drive'/'★個人専用ファイル'/'開発用・個人サービス開発'/'GoogleColab'/

/content/drive/My Drive/★個人専用ファイル/開発用・個人サービス開発/GoogleColab


In [4]:
%ls


data/
duel_dqn_Pendulum-v0_weights.h5f
h5f/
model.txt
playground_deep_learning_keiba
playground_dqn_keiba
playground_dqn_keiba_try_layer_add_depth
playground_dqn_keiba_try_long_term_reward
playground_dqn_keiba_try_simple
playground_dqn_keiba_try_simple2
playground_dqn_keiba_try_simple_x_1
playground_dqn_study.ipynb
playground_dqn_study_my_try.ipynb
playground_dqn_study_my_try_modify_reward
playground.ipynb


In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
from pprint import pprint

In [8]:
!pip install tensorflow==2.2.0

In [0]:
!pip install -q keras

In [10]:
import keras
from keras.utils import np_utils


Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,LeakyReLU
from keras.optimizers import RMSprop


In [0]:
# 競馬データの作成

In [0]:
ldat = pd.read_pickle("./data/keiba_dat_res_20190611_sorted.pkl")

In [14]:
ldat["rank"] 

0          1.0
1          2.0
2          3.0
3          4.0
4          5.0
          ... 
542850     9.0
542851    10.0
542852    11.0
542853    12.0
542854    13.0
Name: rank, Length: 542855, dtype: float64

In [0]:
# 前回の3位以内にマークを付ける
ldat["prev_high_rank"] = 0
ldat.loc[ldat["prev_rank"] <= 3, "prev_high_rank"] = 1

In [0]:
# 3位以内にマークを付ける
ldat["high_rank"] = 0
ldat.loc[ldat["rank"] <= 3, "high_rank"] = 1

In [0]:
# 前回ランクを段階化
ldat["prev_rank_level"] = round(ldat["prev_rank"]  / 4)

In [0]:
# ランクを段階化
ldat["rank_level"] = round(ldat["rank"]  / 4)

In [0]:
# 人気を段階化
ldat["prev_popularity_level"] = round(ldat["prev_popularity"] / 4)

In [0]:
# スピードを出す
ldat["prev_speed"] = ldat["prev_distance"] / ldat["prev_time_float"] 

In [0]:
# スピードを段階化
ldat["prev_speed_level"] =  round(ldat["prev_speed"]*10000000)

In [0]:
sdat = ldat.sample(frac=1).reset_index(drop=True).dropna(0)

In [0]:
# 勝ち馬と負け馬をちょうど半々になるようにする start

In [0]:
winner = sdat.query("prev_high_rank == 1")

In [0]:
loser = sdat.query("prev_high_rank != 1").head(winner.count()[0])

In [0]:
catted = pd.concat([winner,loser])

In [0]:
sampled = catted.sample(frac=1)

In [0]:
# 勝ち馬と負け馬をちょうど半々になるようにする end

In [0]:
# x , yデータを作る start

In [0]:
x = sampled[['frame_num', 'horse_num', 'name', 'sex', 'age', 'amount',
       'jockey', 'odds', 'popularity', 'weight',
       'diff_weight', 'trainer', 'race_num', 'grade', 'ground', 'direction',
        'distance', 'weather', 'condition', 'start_time_float', 'year',
       'month', 'day', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
       'prev_name', 'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
       'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
       'prev_weight', 'prev_diff_weight', 'prev_trainer', 'prev_race_num',
       'prev_grade', 'prev_ground', 'prev_direction',
       'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float',
       'prev_year', 'prev_month', 'prev_day']].dropna(0)

In [0]:
# x = sdat[['frame_num', 'horse_num', 'sex', 'age', 'amount',
#        'odds', 'popularity', 'weight',
#        'diff_weight', 'trainer', 'grade', 'ground', 'direction',
#         'distance', 'weather', 'condition', 'start_time_float', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
#        'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
#        'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
#        'prev_weight', 'prev_diff_weight', 'prev_trainer',
#        'prev_grade', 'prev_ground', 'prev_direction',
#        'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float'
#        ]].dropna(0)

In [0]:
# x = sdat[["odds",'prev_rank','prev_diff_arrival']].dropna(0)

In [0]:
x = sampled[['prev_rank_level',"prev_speed_level"]].dropna(0)

# lastx_tmpは最終評価用に使用、勝ち負けを半々にしていない
lastx_tmp = sdat[['prev_rank_level',"prev_speed_level"]].dropna(0)

In [0]:
#x = sampled[['high_prev_popularity']].dropna(0)

In [0]:
#y = sampled[["high_rank"]].dropna(0)

In [0]:
y_np = np_utils.to_categorical(sampled[["high_rank"]].dropna(0))
y = pd.DataFrame(y_np,columns=["lose","win"],index=sampled.index)

# lasty_tmpは最終評価用に使用、勝ち負けを半々にしていない
lasty_np = np_utils.to_categorical(sdat[["high_rank"]].dropna(0))
lasty_tmp = pd.DataFrame(lasty_np,columns=["lose","win"],index=sdat.index)

In [161]:
len(lasty)

352595

In [162]:
sampled["high_rank"]

74032     0
534112    0
79938     0
441140    1
7727      0
         ..
31590     0
105440    0
189232    0
50804     0
45160     0
Name: high_rank, Length: 208460, dtype: int64

In [0]:
# x , yデータを作る end

In [0]:
# 競馬データの作成 終わり

In [0]:
# 学習、テストデータの作成

In [0]:
trainlen = int(len(x)/2)

In [0]:
trainx = x.head(trainlen)
trainy = y.head(trainlen)

In [0]:
testx = x.tail(len(x)-trainlen)
testy = y.tail(len(x)-trainlen)

In [0]:
testorg = sampled.tail(len(x)-trainlen) 

In [0]:
# 学習、テストデータの作成　終わり

In [171]:
testx.loc[542846]

KeyError: ignored

In [0]:
# 最終検証用データからトレーニングデータを取り除く
lastx = lastx_tmp[~lastx_tmp.index.isin(trainx.index)]
lasty = lasty_tmp[~lasty_tmp.index.isin(trainy.index)]

In [0]:
lastx

In [0]:
lasty

In [0]:
#!pip uninstall -y keras

In [0]:
#!pip uninstall -y tensorflow

In [0]:
batch_size = 128
num_classes = 2
epochs = 20


In [0]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x.shape[1],)))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(256, activation=None))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(128, activation=None))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(2)) # 出力層2ノード,全結合
model.add(Activation("sigmoid"))

In [174]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1536      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


In [0]:
history = model.fit(trainx, trainy,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(testx, testy))


Train on 104230 samples, validate on 104230 samples
Epoch 1/20
104230/104230 [==============================] - 12s 116us/step - loss: 0.5513 - accuracy: 0.7267 - val_loss: 0.5518 - val_accuracy: 0.7257
Epoch 2/20
 73216/104230 [====================>.........] - ETA: 1s - loss: 0.5484 - accuracy: 0.7271

In [0]:
# 予測
predictions = model.predict(lastx)

In [0]:
print(predictions)

In [0]:
# 評価
score = model.evaluate(lastx, lasty, verbose=1)


In [0]:
len(lastx)

In [0]:

print("Test score", score[0])
print("Test accuracy", score[1])

In [0]:
predict = pd.DataFrame(predictions, columns={"lose","win"})

In [0]:
result =  pd.concat([ predict.reset_index(),testorg.reset_index()],axis=1)

In [0]:
result["final_return"] = result["return"]
result.loc[result["return"] == 0, "final_return"] = -1


In [0]:
result.query("win > 0.5").sum()["final_return"]

In [0]:
result.query("win > 0.5").mean()["final_return"]

In [0]:
result.count()

In [0]:
testorg.sum()["return"]

In [0]:
testorg.count()